In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
rcParams['figure.figsize'] = 10,8
import seaborn as sb

In [16]:
train = pd.read_csv('C:/Hari Docs/Dataset/houseprices_train.csv')
test = pd.read_csv('C:/Hari Docs/Dataset/houseprices_test.csv')
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [17]:
#Adding source
train['source'] = 'train'
test['source'] = 'test'
print(train.shape, test.shape)

(1460, 82) (1459, 81)


In [18]:
df = pd.concat([train, test])
df.shape

(2919, 82)

In [19]:
#Removing unwanted columns
del_col = ['Alley','1stFlrSF','2ndFlrSF','Fireplaces','FireplaceQu','LotFrontage','LandContour','BldgType','HouseStyle'
           ,'RoofStyle','RoofMatl','Exterior1st'
           ,'Exterior2nd','MasVnrType','MasVnrArea','ExterCond','Foundation','BsmtCond','BsmtExposure','BsmtFinType1'
           ,'BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','Heating','CentralAir','Electrical','LowQualFinSF'
           ,'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','KitchenAbvGr','Functional','GarageType','GarageYrBlt'
           ,'GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond','PavedDrive','WoodDeckSF','OpenPorchSF'
           ,'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','PoolQC','Fence','MiscFeature','MiscVal','SaleType'
           ,'SaleCondition']
df.drop(del_col, axis=1, inplace=True)
df.shape

(2919, 28)

In [20]:
#All null values of BsmtQual is no Basement
rows = df['BsmtQual'].isnull()
print ('Before: ', df['BsmtQual'].isnull().sum())
df.loc[rows,'BsmtQual'] = 'None'
print ('After: ', df['BsmtQual'].isnull().sum())

Before:  81
After:  0


In [21]:
#TotalBsmtSF = 0 values
zero = (df['BsmtQual'] == 'None')
print ('Before:', df['TotalBsmtSF'].isnull().sum())
df.loc[zero, 'TotalBsmtSF'] = 0
print ('After:', df['TotalBsmtSF'].isnull().sum())

Before: 1
After: 0


In [22]:
#Filling missing values for KitchenQual, MSZoning, Utilities
df['KitchenQual'].fillna(df['KitchenQual'].mode()[0], inplace= True)

df['MSZoning'].fillna(df['MSZoning'].mode()[0], inplace= True)

df['Utilities'].fillna(df['Utilities'].mode()[0], inplace= True)

In [23]:
#Removing few categorical variable because most of them have 90% same value
df.drop(['Condition1', 'Condition2', 'LandSlope', 'Street', 'Utilities'], axis=1, inplace=True)
df.shape

(2919, 23)

In [24]:
rows = df['YearRemodAdd'] > df['YrSold'] 
df.loc[rows,['YearRemodAdd','YrSold','YearBuilt']]

,YearRemodAdd,YrSold,YearBuilt
523,2008,2007,2007
835,2008,2007,2007
1089,2009,2007,2008


In [26]:
df.loc[rows,'YearRemodAdd'] = 2007
df.loc[rows,['YearRemodAdd','YrSold','YearBuilt']]

,YearRemodAdd,YrSold,YearBuilt
523,2007,2007,2007
835,2007,2007,2007
1089,2007,2007,2008


In [28]:
rows = df['YearRemodAdd'] < df['YearBuilt'] 
df.loc[rows,['YearRemodAdd','YrSold','YearBuilt']]

,YearRemodAdd,YrSold,YearBuilt
416,2001,2009,2002
1089,2007,2007,2008


In [30]:
df.loc[rows,'YearRemodAdd'] =df.loc[rows, 'YearBuilt']

In [31]:
df.loc[rows,['YearRemodAdd','YrSold','YearBuilt']]

,YearRemodAdd,YrSold,YearBuilt
416,2002,2009,2002
1089,2008,2007,2008


In [32]:
rows = df['YearRemodAdd'] > df['YrSold'] 
df.loc[rows,['YearRemodAdd','YrSold','YearBuilt']]

,YearRemodAdd,YrSold,YearBuilt
1089,2008,2007,2008


In [34]:
df.loc[rows,'YrSold'] = 2008
df.loc[rows,['YearRemodAdd','YrSold','YearBuilt']]

,YearRemodAdd,YrSold,YearBuilt
1089,2008,2008,2008


In [35]:
df['BlgAge'] = df['YrSold'] - df['YearRemodAdd']

KeyError: 'BldgAge'

0     5
1    31
2     6
3    36
4     8
Name: BlgAge, dtype: int64